#### Problem 1.  10 points

# Wires and gates.

A wire is identified by a Python string, for example 'x', 'y0', or 'next'.

#### A gate is an object with three fields: 
1) a string indicating the type of gate, one of:
   'not', 'and', 'or', 'xor', 'nand', 'nor'
2) a list of the wire identifiers of the inputs
3) the output wire identifier

Below is the (partial) definition of the gate class, followed by
several examples.

Some of the methods are undefined, as designated "pass".  Provide
those definitions.

One of the methods is good_gate, which returns True if it is a
well-formed gate, and False otherwise.

To be a well-formed gate, the value must be a gate object whose three
fields satisfy the following conditions:

(1) the type field is one of the gate strings: 'not', 'and', 'or',
     'xor', 'nand', 'nor',

(2) the inputs field is a list of wire identifiers

(3) the output field is a single wire identifier

In addition, the number of inputs should be correct for each gate
type.  A gate of type 'not' has 1 input, while gates of types 'and',
'or', 'xor', 'nand', 'nor' have 2 inputs.

In [ ]:
class gate:

    def __init__(self, type, inputs, output):
        self.type = type.lower()  ## make all gate types lowercase.
        self.inputs = inputs
        self.output = output

    def __repr__(self):
        return f"gate({self.type!r}, {self.inputs!r}, {self.output!a})"

    def good_gate(self):
        valid_type = ['and', 'or', 'xor', 'nand', 'nor', 'not']

        if self.type not in valid_type:
            return False
        if not isinstance(self.inputs, list):
            return False
        if not isinstance(self.output, str):
            return False
        if len(self.output) > 1 or len(self.output) < 1:
            return False
        else:
            for i in self.inputs:
                    if not isinstance(i, str):
                        return False
            if self.type == "not":
                return len(self.inputs) == 1
            else:
                return len(self.inputs) == 2
                    
    ## the overloaded == operator for gates.
    def __eq__(self, other):
        return (type(self) == type(other) and
                type(self) == gate and
                self.type == other.type and
                set(self.inputs) == set(other.inputs) and
                self.output == other.output)

#### Problem 2. 10 points

# Circuits.

#### A circuit is an object with three fields
 1. a list of input wire identifiers
 2. a list of output wire identifiers
 3. a list of gates

Below is the (partial) definition of the circuit class, followed by
several examples.

Some of the methods are undefined, as designated "pass".  Provide
those definitions.

One of the methods is good_circuit, which returns True if it is a
well-formed circuit, and False otherwise.

To be a well-formed circuit, it must be a circuit object and its
inputs field must be a list of wires, its outputs field must be a list
of wires, and its gates field must be a list of gates that are
well-formed according to the good_gate method.

In addition, the circuit must satisfy the following conditions:

 1.  no input of the circuit is the output of a gate,

 2. every input of a gate is either an input of the circuit or the
 output of a gate,

 3. no wire is the output of two or more gates,

 4. every output of the circuit is either an input of the circuit or
 the output of a gate.

In [ ]:
class circuit:
    
    def __init__(self, inputs, outputs, gates):
        self.inputs = inputs
        self.outputs = outputs
        self.gates = gates

    def __repr__(self):
        return f"circuit({self.inputs!r}, {self.outputs!r}, {self.gates!r})"

    def good_circuit(self):
        if not gate(self):
            return False
        if not self.inputs == self.outputs:
            return True

        for g in self.gates:
            if not g.good_gate():
                return False
        
        g_outputs = [g.outputs for g in self.gates]
        g_inputs = [i for g in self.gates for i in g.inputs]

        if g_outputs in g_inputs:
            return False
        
        if self.outputs in g_outputs:
                return False
        if self.outputs in g_outputs or self.outputs in self.inputs:
            return True

    ## See problem 3 below
    def all_wires(self):
        pass

    ## See problem 3 below
    def find_gate(self, wire):
        pass

    ## See problem xx below.  You do not have to write this one.
    def next_value(self, wire, config):
        if wire in self.inputs:
            return config[wire]
        wgate = self.find_gate(wire)
        wtype = wgate.type
        winputs = wgate.inputs
        input_values = list(map(lambda x: config[x], winputs))
        
        if wtype == 'not':
            return 1 - input_values[0]
        if wtype == 'and':
            if input_values[0] == 0:
                return 0
            else:
                return input_values[1]
        if wtype == 'or':
            if input_values[0] == 1:
                return 1
            else:
                return input_values[1]
        if wtype == 'xor':
            if input_values[0] == input_values[1]:
                return 0
            else:
                return 1
        if wtype == 'nand':
            if input_values[0] == 0:
                return 1
        else:
                return 1 - input_values[1]
        if wtype == 'nor':
            if input_values[0] == 0:
                return 0
            else:
                return 1 - input_values[1]

    ## See problem 5 below
    def next_config(self, config):
        pass

    ## See problem 6 below
    def stable(self,config):
        pass

    ## See problem 6 below
    def all_stable_configs(self):
        pass

    ## See problem 6 below
    def output_values(self,config):
        pass

    ## See problem 6 below
    def init_config(self, input_values):
        pass
        
    ## See problem 7 below
    def simulate(self, config, n):
        pass

    ## See problem 8 below    
    def final_config(self,config):
### use in all_stable_configs above
## similar to power_set() in hw1
def all_configs(wires):
    lst = all_configs_aux(wires)
    r = []
    for x in lst:
        r.append({z[0] : z[1] for z in x})
    return r

def all_configs_aux(wires):
    if not wires:
        return [[]]
    ac = all_configs_aux(wires[1:])
    left =  [[[wires[0], 0]] + x for x in ac]
    right =  [[[wires[0], 1]] + x for x in ac]
    return left + right


## We provide the flatten() function which may be useful in defining the circuit methods.

def flatten(tree):
    result = []
    if not isinstance(tree, list):
        return [tree]
    else:
        for x in tree:
            result.extend(flatten(x))
    return result


In [ ]:
eq1_ckt = circuit(["x", 'y'], ['z'],
                  [gate('not', ['x'], 'cx'),
                   gate('not', ['y'], 'cy'),
                   gate('and', ['x', 'y'], 't1'),
                   gate('and', ['cx', 'cy'], 't2'),
                   gate('or', ['t1', 't2'], 'z')])


nullckt = circuit([], [], [])
badckt1 = circuit([1], [2], [3])
badckt2 = circuit([], [], [gateb1])

eq2_ckt = circuit(["x", 'y'], ['z'],
                  [gate('xor', ['x', 'y'], 'w'),
                   gate('not', ['w'], 'z')])
sel_ckt = circuit(["x1", 'x0', 'y1', 'y0', 's'], 
                  ['z1', 'z0'],
                  [gate('not', ['s'], 'sc'),
                   gate('and', ['x1', 'sc'], 'u1'),
                   gate('and', ['y1', 's'], 'v1'),
                   gate('or', ['u1', 'v1'], 'z1'),
                   gate('and', ['x0', 'sc'], 'u0'),
                   gate('and', ['y0', 's'], 'v0'),
                   gate('or', ['u0', 'v0'], 'z0')])
seq_or_ckt = circuit(["x"], 
                     ['z'],
                     [gate('or', ['x', 'z'], 'z'),])
clock_ckt = circuit([], 
                    ['z'],
                    [gate('not', ['z'], 'z'),])